<a href="https://colab.research.google.com/github/TimeryaevaM/new_rep/blob/master/diplom_3v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [649]:
import pandas as pd
import os
import csv
import numpy as np

In [650]:
# df = pd.read_csv("/data_new.csv", sep=",", quoting=csv.QUOTE_ALL, encoding='utf-8')

df = pd.read_csv("/data_new.csv")

In [651]:
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [652]:
df.shape

(377185, 18)

In [653]:
df[df['target'].isna()].shape

(2481, 18)

In [654]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [655]:
print("Percent NULL values\n")
round(df.isna().sum() / df.shape[0] * 100, 2)

Percent NULL values



status          10.58
private pool    98.89
propertyType     9.21
street           0.00
baths           28.19
homeFacts        0.00
fireplace       72.66
city             0.01
schools          0.00
sqft            10.76
zipcode          0.00
beds            24.20
state            0.00
stories         39.96
mls-id          93.39
PrivatePool     89.31
MlsId           17.73
target           0.66
dtype: float64

Удаляем столбцы, где процент NULL значений больше 40%

In [656]:
df.drop(['private pool', 'fireplace', 'mls-id', 'PrivatePool'], axis=1, inplace=True)

In [657]:
df.nunique()

status             159
propertyType      1280
street          337076
baths              229
homeFacts       321009
city              2026
schools         297365
sqft             25405
zipcode           4549
beds              1184
state               39
stories            348
MlsId           232944
target           43939
dtype: int64

status

In [658]:
df['status'] = df['status'].str.lower()

In [659]:
import string
string.punctuation
string.digits

'0123456789'

In [660]:
df['status'] = df['status'].str.replace('[{}]'.format(string.punctuation), ' ').str.replace('[{}]'.format(string.digits), ' ').apply(lambda x: " ".join(str(x).split()))
df['status'].value_counts()

for sale                   199571
active                     105207
nan                         39918
foreclosure                  6769
new construction             5475
                            ...  
pending backups wanted          1
contingent sale of home         1
pending take backups            1
conting accpt backups           1
under contract w bckp           1
Name: status, Length: 104, dtype: int64

In [661]:
df['status'].nunique()

104

propertyType 

In [662]:
df['propertyType'] = df['propertyType'].str.lower()
df['propertyType'] = df['propertyType'].str.replace('[{}]'.format(string.punctuation), ' ').apply(lambda x: " ".join(str(x).split()))
df['propertyType'] = df['propertyType'].str.replace('one', '1').str.replace('two', '2')

In [663]:
df['propertyType'].value_counts()[20:40]

contemporary modern           1010
apartment                      922
2 stories                      911
mfd mobile home                792
single detached                751
detached 2 story               638
2 story                        596
transitional                   560
cooperative                    535
florida                        530
bungalow                       442
farms ranches                  371
garden home                    305
cape cod                       281
2 stories traditional          280
spanish mediterranean          272
other style                    249
1 story traditional            238
other                          232
condo townhome traditional     225
Name: propertyType, dtype: int64

In [664]:
df['stories_dop'] = df['propertyType'].str.findall(r'[a-z0-9]+ story|[a-z0-9]+ stories').apply(lambda x: None if len(x) == 0 else x[-1])
#.str.replace('one', '1').str.replace('two', '2')
df['stories_dop'].value_counts(dropna=False)

NaN          369527
1 story        4440
2 stories      1558
2 story        1466
3 stories       134
8 stories        38
7 stories        15
3 story           6
5 story           1
Name: stories_dop, dtype: int64

In [665]:
set_story = set(df['stories_dop'].dropna())
for y in set_story:
  df['propertyType'] = df['propertyType'].str.replace(y, '')

In [666]:
df[df['propertyType'].str.contains('home')]['propertyType'].value_counts()

single family home                                           123934
condo townhome row home co op                                  7701
multi family home                                              1501
mfd mobile home                                                 792
garden home                                                     305
                                                              ...  
condo townhome spanish                                            1
condo townhome a frame traditional                                1
single detached vacation home                                     1
designated historical home colonial traditional victorian         1
apartment condo townhome garden zero lot line                     1
Name: propertyType, Length: 137, dtype: int64

In [667]:
df['propertyType'] = df['propertyType'].str.replace('home', '')
df['propertyType'] = df['propertyType'].apply(lambda x: None if x == '' else  " ".join(str(x).split()))

In [668]:
df['propertyType'].nunique()

1113

street

In [669]:
df['street'].value_counts(dropna=False)[1200:]

2964 Rusk St               3
3617 W 155th St            3
18 Hermann Park Ct         3
3124 W 88th St             3
7210 Granvia Dr            3
                          ..
101 Mulberry St APT 220    1
34 S Grande Pointe Dr      1
12720 Banting Ter          1
12233 Pheon St             1
821 Kodak Dr               1
Name: street, Length: 335877, dtype: int64

In [670]:
df['street'] = df['street'].str.lower()
df['street'] = df['street'].apply(lambda x: None if 'not disclosed' in str(x) else 
                                  None if 'undisclosed' in str(x) else
                                  None if 'not available' in str(x) else
                                  None if 'unknown' in str(x) else str(x))

In [671]:
df.loc[(df['street'].isna()==False) & (df['street'].str.contains(r'mls ')), 'street'] = None

In [672]:
df.loc[81361, 'street']

'1131 w 24th street st unit c '

In [673]:
punct = string.punctuation
df['street'] = df['street'].str.replace('[{}]'.format(string.punctuation), ' ').str.replace(r'\b[0-9]+\b', '').str.replace('  ', ' ').str.strip()

In [674]:
street_list_tuple = [('st', 'street'), ('dr', 'drive'), ('ave', 'avenue'), ('rd', 'road'), ('rdg', 'ridge'), ('blvd', 'boulevard'), ('pkwy', 'parkway'), 
                   ('cir', 'circle'), ('hwy', 'highway'), ('ln', 'lane'), ('lndg', 'landing'), ('trl', 'trail'), ('ter', 'terrace'), ('ct', 'court'), 
                   ('crk', 'creek'), ('pl', 'place'), ('rds', 'roads'), ('bnd', 'bend'), ('fls', 'falls'), ('clf', 'cliff'), ('clfs', 'cliff'), ('mtn', 'mountain'), 
                   ('crst', 'crest'), ('hl', 'hill'), ('ml', 'mill'), ('mnr', 'manor'), ('hts', 'heights')]       
                          

In [675]:
def street_replace(df, st_tupl):

  # Функция замены коротких синонимов на полные названия и удаление дубликатов

  df['street'] = df['street'].str.replace(rf'\b{st_tupl[0]}\b', st_tupl[1]).str.replace(rf'\b{st_tupl[1]}\b \b{st_tupl[1]}\b', st_tupl[1])
  df['street'] = df['street'].str.replace('  ', ' ').str.strip()

  # print(f'replace {st_tupl[0]} to {st_tupl[1]}')

In [676]:
for x in street_list_tuple:
  street_replace(df, x)

In [677]:
def del_dropwords_street(df, dropword):

  # Удаляем dropword из признака street
  df['street'] = df['street'].str.replace(rf'\b{dropword}\b', '').str.replace(r'  ', ' ').str.strip()

  print(f"Drop {dropword}")

In [678]:
dropwords = ['lot', 'lots', 'collections', 'collection', 'plan', 'apt', 'unit', 'buildable', 'the']

for x in dropwords:
  del_dropwords_street(df, x)

Drop lot
Drop lots
Drop collections
Drop collection
Drop plan
Drop apt
Drop unit
Drop buildable
Drop the


In [679]:
# Удаляем мусор из строк, чтобы легче паттерны было выделять
df['street'] = df['street'].str.replace(r'\b[a-z]{1,2}\b', '').str.replace('  ', ' ').str.strip()
df['street'] = df['street'].str.replace(r'\b[0-9]+[a-z]\b', '').str.replace(r'\b[a-z]+[0-9]\b', '').str.replace(r'\b[0-9]+ft\b', '')
df['street'] = df['street'].str.replace(r'  ', ' ').str.strip()

In [680]:
df['street'] = df['street'].apply(lambda x: None if x is None else
                                  None if x.strip() == '' else x)

In [681]:
df['street'].sample(10)

282858    swift valley ranch express
370303             stibbs cross road
342715              delmar boulevard
55358          brookhurst street spc
233578             belvoir boulevard
291353             forest dale drive
374140               horse trap lane
157731                    3rd street
264661                cocoanut drive
340253                       highway
Name: street, dtype: object

In [682]:
# tokenizer

# ! pip install tokenizer

from keras.preprocessing.text import Tokenizer

In [683]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 10000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 10

In [684]:
street_nlp = df[df['street'].isna() == False]['street'].copy()

In [685]:
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(street_nlp)

In [686]:
# tokenize.word_index

# Токенайзер введем позже

baths

In [687]:
df['baths'] = df['baths'].apply(lambda x: None if x is np.nan else x)
df['baths'].value_counts(dropna=False)

NaN                      106338
2 Baths                   52466
3 Baths                   35506
2                         20452
2.0                       16576
                          ...  
2.25 ba                       1
1.8 Baths                     1
2-1 / 2-1 / 1-1 / 1-1         1
23                            1
5.25 Baths                    1
Name: baths, Length: 230, dtype: int64

In [688]:
punct = '~—\-:'

In [689]:
df['baths'] = df['baths'].str.lower().str.replace('[a-z]+\.?', '').str.replace(',', '.').str.replace('[{}]'.format(punct), ' ')
df['baths'] = df['baths'].apply(lambda x: None if str(x).strip() =='' else x)

In [690]:
df['baths_new'] = df['baths'].str.findall('[0-9]+\.?[0-9]*')
df['baths_new'] = df['baths_new'].apply(lambda x: None if x is None else x[0]).astype(float)
df['baths_new'].value_counts()

# df['baths_new'].to_csv('out.csv')
# ванна 0.5 проверить

2.0     103519
3.0      66683
4.0      26333
1.0      17787
2.5      13786
         ...  
1.8          1
64.0         1
14.5         1
0.5          1
42.0         1
Name: baths_new, Length: 84, dtype: int64

city

In [691]:
df['city'] = df['city'].str.lower().str.strip()
df['city'] = df['city'].str.replace('\-', ' ')

In [692]:
df['city'] = df['city'].str.replace(' city', '').str.replace(' town+[a-z]*', '')
df['city'] = df['city'].str.replace('st\.', 'saint').str.replace(' twp.', '').str.replace('e. ', '').str.replace('ft.', 'fort').str.replace('s.a.', 'san antonio')
df['city'] = df['city'].str.replace(' twp', '').str.replace(' vlg', ' village')
df['city'] = df['city'].str.replace(' hts', 'heights').str.replace(' pt', 'point').str.replace('pt ', 'port ')
df['city'] = df['city'].str.replace('^st ', 'saint ')

In [693]:
df.loc[(df['city'].isna()==False) & (df['city'].str.contains('san antonio')), 'city'] = 'san antonio'

In [694]:
df['city'] = df['city'].str.replace('p c ', 'panama ').str.replace('^n ', 'north ').str.replace("town 'n' country", 'town north country')
df['city'] = df['city'].str.replace('nyork', 'new york').str.replace('nyc', 'new york')
df['city'] = df['city'].str.replace('bed-stuy', 'brooklyn').str.replace('collegepoint', 'college point')
df['city'] = df['city'].str.replace('othvalue [ a-z]*', 'othvalue')

In [695]:
df['city'] = df['city'].apply(lambda x: " ".join(str(x).split()))

In [696]:
df['city'].value_counts()

san antonio        30033
houston            24442
miami              15528
jacksonville       10032
dallas              8858
                   ...  
hollywood hills        1
downtown pgh           1
sand lake              1
plato center           1
prairie grove          1
Name: city, Length: 1785, dtype: int64

sqft

In [697]:
df['sqft'].value_counts()

0             11854
1,200 sqft      839
1,000 sqft      654
1,100 sqft      573
1,800 sqft      563
              ...  
35 sqft           1
9,893             1
5,825             1
4664              1
7,049 sqft        1
Name: sqft, Length: 25405, dtype: int64

In [698]:
df['sqft'] = df['sqft'].apply(lambda x: None if x is np.nan else x)

In [699]:
df['sqft'] = df['sqft'].str.replace('[a-z ]*: ', '')
df['sqft'] = df['sqft'].str.replace('-','').str.replace('[A-Za-z]+','').str.replace(',','')

In [700]:
df['sqft'] = df['sqft'].str.strip().apply(lambda x: None if x is None else 
                                          None if x == '' else int(x))

In [701]:
df['sqft'].value_counts()

0.0         11854
1200.0       1426
1000.0       1027
1500.0       1014
1800.0        988
            ...  
8966.0          1
11089.0         1
123275.0        1
44351.0         1
32767.0         1
Name: sqft, Length: 9897, dtype: int64

zipcode

In [702]:
df['zipcode'].value_counts()

32137         2141
33131         1563
34747         1488
78245         1390
34759         1333
              ... 
3316             1
27712-3137       1
77613            1
27617-8314       1
79755            1
Name: zipcode, Length: 4549, dtype: int64

In [703]:
df['zipcode'] = df['zipcode'].str.replace('\-[0-9]*', '').apply(lambda x: None if x is None else None if x == '' else int(x))

beds

In [704]:
df['beds'] = df['beds'].apply(lambda x: None if x is np.nan else x).str.lower()

In [705]:
df['beds'].value_counts(dropna=False)

NaN            91282
3 beds         53459
4 beds         35418
3              31406
2 beds         26362
               ...  
540 sqft           1
2.71 acres         1
1,385 sqft         1
10,367 sqft        1
8,023 sqft         1
Name: beds, Length: 1185, dtype: int64

In [706]:
index_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('sqft')), ['sqft','beds']].index
new_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('sqft')), ['sqft','beds']]
new_sqft['sqft'] = new_sqft['beds'].str.strip().str.replace(',', '.').str.findall('[0-9]+\.*[0-9]*').apply(lambda x: None if len(x) == 0 else x[0]).astype(float)

df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('sqft')), 'sqft'] = new_sqft['sqft']
df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('sqft')), 'beds'] = None

In [707]:
index_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('acre')), ['sqft','beds']].index
new_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('acre')), ['sqft','beds']]
new_sqft['sqft'] = new_sqft['beds'].str.strip().str.replace(',', '.').str.findall('[0-9]+\.*[0-9]*').apply(lambda x: None if len(x) == 0 else x[0]).astype(float)
new_sqft['sqft'] = (new_sqft['sqft'] * 43560).astype(int)

df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('acre')), 'sqft'] = new_sqft['sqft']
df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('acre')), 'beds'] = None

In [708]:
df['beds'] = df['beds'].str.replace('bedrooms', 'beds').str.replace('bd', 'beds')

In [709]:
df.loc[(df['beds'].isna()==False) & (df['beds'].str.strip().str.contains('^[a-z,\-%@#_:;~ ]+$')), 'beds'] = None

In [710]:
df['beds'].value_counts(dropna=False)[:20]

NaN       111390
3 beds     66336
4 beds     43683
2 beds     31605
3          31406
4          20030
2          16110
5 beds     13904
3.0         8088
5           6424
4.0         5231
1           4610
6 beds      4466
2.0         2430
6           1806
5.0         1372
7 beds      1247
0           1230
1 beds      1081
8 beds       856
Name: beds, dtype: int64

In [711]:
new_bath = df.loc[(df['baths_new'].isna()) & (df['beds'].str.contains('bath')), 'beds'].str.findall('[0-9a-z ]+ bath').apply(lambda x: x[0].split()[0]).astype(float)
df.loc[new_bath.index, 'baths_new'] = new_bath
df.loc[new_bath.index, 'baths_new']

18787     1.0
124819    1.0
172605    1.0
238222    1.0
279149    2.0
325249    1.0
Name: baths_new, dtype: float64

In [712]:
new_beds = pd.concat([(df.loc[(df['beds'].isna()==False) & (df['beds'].str.strip().str.contains('[0-9 a-z]+bed[a-z]*,')), 'beds']),
                      df.loc[(df['beds'].isna()==False) & (df['beds'].str.strip().str.contains(',[0-9 a-z]+bed[a-z]*')), 'beds']
                      ],
                     ).reset_index()
new_beds = new_beds.drop_duplicates().set_index('index')
new_beds['beds'] = new_beds['beds'].str.replace('or more ', '').str.findall('[0-9 ]+bed').apply(lambda x: x[0].split()[0]).astype(float)

df.loc[new_beds.index, 'beds'] = new_beds['beds']
df.loc[new_beds.index, 'beds']

index
18787     2
172605    2
200331    3
238222    3
325249    2
124819    2
279149    3
Name: beds, dtype: object

In [713]:
punct = '!@#_,:;&'
df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('[{}]'.format(punct))), 'beds'] = None
punct = '\-'
new_beds = df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('[{}]'.format(punct))), 'beds'].str.findall('[0-9]+').apply(lambda x: x[-1]).astype(float)
df.loc[new_beds.index, 'beds'] = new_beds
df.loc[new_beds.index, 'beds']

330468    2
Name: beds, dtype: object

In [714]:
df['beds'] = df['beds'].str.replace('or more ', '')
df['beds'] = df['beds'].str.replace('beds','').str.strip().apply(lambda x: None if x is None else
                                                                 None if x == '' else x).astype(float)

state

In [715]:
df['state'].value_counts(dropna=False)[-20:]

UT    1325
MD    1090
VT     868
MO     866
VA     801
WI     452
NJ     436
ME     259
IA     242
KY      90
OK      49
MS      40
SC      28
MT       7
DE       5
OS       1
AL       1
Fl       1
OT       1
BA       1
Name: state, dtype: int64

stories

In [716]:
df['stories_dop'] = df['stories_dop'].apply(lambda x: None if x is None else x.split()[0]).astype(float)
df['stories_dop'].value_counts(dropna=False)

NaN    369527
1.0      4440
2.0      3024
3.0       140
8.0        38
7.0        15
5.0         1
Name: stories_dop, dtype: int64

In [717]:
df['stories'] = df['stories'].str.lower()
df['stories'].value_counts(dropna=False)

NaN                                150715
1.0                                 67454
2.0                                 55283
1                                   23086
2                                   18146
                                    ...  
65.0                                    1
2.20                                    1
t3-townhouse 3+ stories                 1
one and one half, three or more         1
ground level, two                       1
Name: stories, Length: 349, dtype: int64

In [718]:
punct = '!|/_\-:;'
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('[{}]'.format(punct))), 'stories'].value_counts()
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('[0-9 ]/[0-9 ]')), 'stories'].value_counts()
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('half')), 'stories'].value_counts()
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('multi')), 'stories'].value_counts()

# много пустых значений, не знаю, обрабатывать ли их

multi/split                   271
multi-level                    57
other multi-unit               19
three or more, multi/split      7
multi level                     6
multi-story                     3
two, multi/split                3
multi level, three or more      1
2 story, multi level            1
multi-level, new englander      1
multi/split, two                1
one, two, multi/split           1
other, two, multi/split         1
tri/multi level                 1
1 story, multi level            1
3 story, multi level            1
multi level, tri level          1
Name: stories, dtype: int64

MlsId

In [719]:
df = df.drop('MlsId', axis=1)

homeFacts

In [720]:
df['homeFacts'] = df['homeFacts'].apply(lambda x: eval(x))

In [721]:
df['homeFacts_new'] = df['homeFacts'].apply(lambda x: list(x.values())[0])

In [722]:
list_columns = ['Year built', 'Remodeled year', 'Heating', 'Cooling', 'Parking', 'lotsize', 'Price/sqft']

for column in enumerate(list_columns):
  df[column[1]] = None
  df[column[1]] = df['homeFacts_new'].apply(lambda x: None if x[column[0]].get('factValue') == '' else  x[column[0]].get('factValue'))

Year built

In [723]:
df['Year built'].value_counts(dropna=False)

NaN     62374
2019    32171
2006     8008
2005     7503
2007     7152
        ...  
1735        1
1208        1
1804        1
1809        1
1807        1
Name: Year built, Length: 230, dtype: int64

In [724]:
df['Year built'] = df['Year built'].apply(lambda x: None if x is None else
                                          None if 'No' in x else int(x))

In [725]:
df['Year built'].value_counts(dropna=False)

NaN       63596
2019.0    32171
2006.0     8008
2005.0     7503
2007.0     7152
          ...  
1805.0        1
1807.0        1
1808.0        1
1809.0        1
1019.0        1
Name: Year built, Length: 229, dtype: int64

Remodeled year

In [726]:
df['Remodeled year'].value_counts(dropna=False)

# Слишком много пропусков, удалим потом столбец

NaN     226110
2006      5538
2005      4817
2007      4387
2008      3796
         ...  
1888         1
1853         1
1845         1
1870         1
1738         1
Name: Remodeled year, Length: 154, dtype: int64

Heating

In [727]:
df['Heating'].value_counts(dropna=False)[:10]

# Слишком много пропусков и непоказательных данных, удалим потом столбец

NaN                 109332
Forced Air           82807
Forced air           51506
Other                29623
Electric             10211
Gas                   9296
No Data               8611
Central Air           7814
Central Electric      7112
Central               6247
Name: Heating, dtype: int64

Cooling

In [728]:
df['Cooling'].value_counts(dropna=False)

# Слишком много пропусков и непоказательных данных, удалим потом столбец

Central                                                                                         158754
NaN                                                                                             123967
Central Air                                                                                      14384
No Data                                                                                          10616
Has Cooling                                                                                       9730
                                                                                                 ...  
Gas (Hot Air), Window Unit                                                                           1
2 Air Conditioners                                                                                   1
Central Air, Electric Hot Air, Gas (Hot Air), Heat Pump for A/C, Heat Pump for Heat, Propane         1
Central Air, Heat Pump - AC, Humidifier, Multizone A/C, Natural Gas      

Parking

In [729]:
df['Parking'].value_counts(dropna=False)[:10]

# Слишком много пропусков и непоказательных данных, удалим потом столбец

NaN                         175420
Attached Garage              70752
2 spaces                     28063
1 space                      14252
No Data                      13334
Detached Garage              13201
Carport                       7743
3 spaces                      4724
Off Street                    3622
Carport, Attached Garage      3025
Name: Parking, dtype: int64

lotsize sqft

In [730]:
df[['lotsize', 'sqft']].sample(5)

,lotsize,sqft
175667,"10,200 Sq. Ft.",NaN
13556,0.33 acres,1600.0
324435,"4,064 sqft",2386.0
374364,"5,663 sqft",2087.0
130805,None,1212.0


In [731]:
df['lotsize'] = df['lotsize'].str.lower().str.replace(r'[\-,—]', '').str.strip().apply(lambda x: None if x is None else
                                                                                      None if 'no' in x else
                                                                                      None if x == '' else x)

In [732]:
# Заполним те значения признака sqft которые пустые из признака lotsize

# acres
new_lot_size = df.loc[(df['sqft'].isna()) & (df['lotsize'].isna()==False) & (df['lotsize'].str.contains('sq'))]['lotsize']
new_lot_size = new_lot_size.str.lower().str.replace(r'[a-z \.]', '').str.strip()
new_lot_size = new_lot_size.apply(lambda x: None if x is None else 
                                  None if x is np.nan else
                                  None if x == '' else int(float(x)))
df.loc[new_lot_size.index, 'sqft'] = new_lot_size

# acres
new_lot_size = df.loc[(df['sqft'].isna()) & (df['lotsize'].isna()==False) & (df['lotsize'].str.contains('acre'))]['lotsize']
new_lot_size = new_lot_size.str.lower().str.replace('[a-z ]', '').str.strip()
new_lot_size = new_lot_size.apply(lambda x: None if x is None else float(x) * 43560).astype(int)
df.loc[new_lot_size.index, 'sqft'] = new_lot_size

# Другие!

In [733]:
df['sqft'].isna().sum()

11890

Price/sqft

In [734]:
df['Price/sqft'].sample(10)

194476              $186
29568               None
265085              None
240838         $109/sqft
167384              $128
56389     $151 / Sq. Ft.
33426               $177
143295              $192
50540                $84
326226              $155
Name: Price/sqft, dtype: object

In [735]:
df['Price/sqft'] = df['Price/sqft'].str.lower().str.replace(r'[,/$a-z\. ]', '').str.strip().apply(lambda x: None if x is None else
                                                                                      None if x == '' else int(x))

schools

In [736]:
df['schools'] = df['schools'].apply(lambda x: eval(x)[0])

In [737]:
list_keys = ['data', 'name', 'rating']
# name не будем брать

list_columns = ['Distance', 'Grades', 'rating']
# Grades не будем брать слишком запутано

df['Distance'] = df['schools'].apply(lambda x: None if x.get('data').get('Distance') is None else x.get('data').get('Distance'))
df['rating'] = df['schools'].apply(lambda x: None if x.get('rating') is None else x.get('rating'))

In [738]:
df['Distance'] = df['Distance'].apply(lambda x: [float(y.replace('mi', '').strip()) for y in x])

In [739]:
df['Distance'] = df['Distance'].apply(lambda x: list(filter(None, x)))

In [742]:
df['Distance'].sample(10)

144401                                      [0.29]
257921                          [1.59, 1.21, 0.26]
163861                             [0.2, 0.5, 0.9]
369642                          [1.95, 1.55, 1.22]
61178     [0.5, 0.5, 2.4, 0.7, 1.5, 1.8, 1.3, 1.5]
102041                          [4.42, 5.96, 0.32]
198176            [0.41, 1.49, 1.41, 33.16, 27.64]
371195              [1.0, 4.5, 1.9, 4.7, 1.9, 9.0]
59771     [0.5, 2.6, 1.0, 0.7, 1.1, 1.0, 0.8, 1.5]
346625                             [0.7, 0.5, 1.3]
Name: Distance, dtype: object

In [743]:
df['rating'] = df['rating'].apply(lambda x: [None if "NR" in str(y) else
                                             None if "None" in str(y) else
                                             None if "NA" in str(y) else
                                             None if str(y) == '' else
                                             int(str(y).split('/')[0]) if "/" in str(y) else
                                             int(y) for y in x])

In [744]:
df['rating'] = df['rating'].apply(lambda x: list(filter(None, x)))

In [745]:
df['rating'].sample(10)

111864                                           [10, 5, 7]
170743                                            [4, 3, 2]
226740                                            [5, 7, 5]
184055                                            [5, 7, 6]
172608    [4, 2, 3, 2, 3, 6, 2, 2, 5, 5, 5, 4, 3, 5, 4, ...
79842                                             [5, 5, 9]
172669                                            [3, 6, 7]
204512                                            [5, 6, 7]
293744                                            [2, 4, 3]
357097                                            [2, 3, 2]
Name: rating, dtype: object

target

In [626]:
df['target'].value_counts(dropna=False)

NaN           2481
$225,000      1462
$275,000      1355
$250,000      1312
$350,000      1296
              ... 
$418,464         1
$54,472          1
18,800,000       1
$372,078         1
$3,396,000       1
Name: target, Length: 43940, dtype: int64

In [627]:
df['target'] = df['target'].apply(lambda x: None if x is np.nan else x)

In [628]:
df[(df['target'].isna()==False) & (df['target'].str.isdigit())]['target'].value_counts()

800    2
3      2
1      2
825    1
20     1
29     1
8      1
750    1
25     1
30     1
Name: target, dtype: int64

In [629]:
punct = '$,+\-/'
df['target'] = df['target'].str.replace('[a-z]', '').str.replace('[{}]'.format(punct), '').str.strip()

In [630]:
df['target'] = df['target'].apply(lambda x: None if x is None else int(x.split()[0]))

In [631]:
df.loc[330468]['target']

1215.0

In [632]:
df['target'].value_counts(dropna=False)

NaN         2481
225000.0    1806
275000.0    1650
250000.0    1644
350000.0    1641
            ... 
9100.0         1
515530.0       1
195050.0       1
780226.0       1
374521.0       1
Name: target, Length: 34233, dtype: int64

target Price/sqft

In [633]:
# Заполним те значения признака sqft которые пустые из признака Price/sqft и признака target

new_lot_size = df.loc[(df['sqft'].isna()) & (df['target'].isna()==False) & (df['Price/sqft'].isna()==False)][['Price/sqft', 'target']]
new_lot_size['sqft'] = (new_lot_size['target'] / new_lot_size['Price/sqft']).astype(int)
df.loc[new_lot_size.index, 'sqft'] = new_lot_size['sqft']

# Заполним те значения признака Price/sqft которые пустые из признака sqft и признака target

new_prsq = df.loc[(df['sqft'].isna()==False) & (df['target'].isna()==False) & (df['Price/sqft'].isna())][['sqft', 'target']]
new_prsq['Price/sqft'] = round(new_prsq['target'] / new_prsq['sqft'])
df.loc[new_prsq.index, 'Price/sqft'] = new_prsq['Price/sqft']

future engeneer

In [634]:
df.sample(5)

,status,propertyType,street,baths,homeFacts,city,schools,sqft,zipcode,beds,state,stories,target,stories_dop,baths_new,homeFacts_new,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft,Distance,rating
158269,for sale,condo,peachtree road,4.0,"{'atAGlanceFacts': [{'factValue': '2007', 'fac...",atlanta,"{'rating': ['5/10', '6/10', '6/10'], 'data': {...",2579.0,30305.0,3.0,GA,2,1350000.0,NaN,4.00,"[{'factValue': '2007', 'factLabel': 'Year buil...",2007.0,None,Forced air,Central,4 spaces,None,523.0,"[0.4, 1.3, 1.3]","[5, 6, 6]"
209481,for sale,townhouse,plateau place,3.5,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",atlanta,"{'rating': ['6/10', '7/10', '6/10'], 'data': {...",2643.0,30309.0,2.0,GA,NaN,775000.0,NaN,3.50,"[{'factValue': '', 'factLabel': 'Year built'},...",NaN,None,None,None,None,None,293.0,"[5.57, 1.9, 2.66]","[6, 7, 6]"
301427,active,nan,cleveland street,1.750,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",houston,"{'rating': ['3', '9', '3', '5', '3'], 'data': ...",NaN,77019.0,NaN,TX,NaN,145000.0,NaN,1.75,"[{'factValue': '', 'factLabel': 'Year built'},...",NaN,None,None,None,None,None,NaN,"[0.2, 0.4, 0.7, 1.0, 2.9]","[3, 9, 3, 5, 3]"
52975,active,nan,spring creek road,None,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",chattanooga,"{'rating': ['3', '4', '2'], 'data': {'Distance...",1079.0,37411.0,NaN,TN,1 level,104900.0,NaN,NaN,"[{'factValue': '1950', 'factLabel': 'Year buil...",1950.0,1976,"Central, Electric","Central, Electric",None,8540 sq. ft.,97.0,"[0.9, 1.9, 2.0]","[3, 4, 2]"
141846,for sale,condo,69th street,11,"{'atAGlanceFacts': [{'factValue': '1926', 'fac...",new york,"{'rating': ['10/10', '3/10', '8/10'], 'data': ...",6241.0,10021.0,4.0,NY,17.0,38500000.0,NaN,11.00,"[{'factValue': '1926', 'factLabel': 'Year buil...",1926.0,2008,Baseboard,Solar,Carport,None,6169.0,"[0.5, 1.6, 2.8]","[10, 3, 8]"


In [635]:
df = df.drop(['stories_dop', 'homeFacts', 'schools', 'stories_dop', 'baths_new', 'homeFacts_new'], axis=1)

In [748]:
round(df.isna().sum() / df.shape[0] * 100, 1)

status             0.0
propertyType       0.9
street             1.9
baths             28.6
homeFacts          0.0
city               0.0
schools            0.0
sqft               3.2
zipcode            0.0
beds              29.7
state              0.0
stories           40.0
target             0.7
stories_dop       98.0
baths_new         28.6
homeFacts_new      0.0
Year built        16.9
Remodeled year    59.9
Heating           29.0
Cooling           32.9
Parking           46.5
lotsize           24.4
Price/sqft        17.5
Distance           0.0
rating             0.0
dist_max           1.1
dist_min           1.1
dist_mean          1.1
rate_max           1.4
rate_min           1.4
rate_mean          1.4
dtype: float64

In [637]:
df = df.drop(['lotsize'], axis=1)
df = df.drop(['stories', 'Remodeled year', 'Heating', 'Cooling', 'Parking'], axis=1)

In [638]:
df = df.drop(['baths', 'beds', 'Year built'], axis=1)

In [639]:
df[['status', 'propertyType', 'city', 'zipcode', 'state']].nunique()

status           104
propertyType    1113
city            1785
zipcode         4267
state             39
dtype: int64

In [746]:
# Содаем новые признаки

import numpy as np

df['dist_max'] = df['Distance'].apply(lambda x: None if x is None else 
                                      None if len(x) == 0 else max(x))
df['dist_min'] = df['Distance'].apply(lambda x: None if x is None else 
                                      None if len(x) == 0 else min(x))
df['dist_mean'] = df['Distance'].apply(lambda x: None if x is None else 
                                      None if len(x) == 0 else np.mean(list(filter(None, x))))

In [747]:
df['rate_max'] = df['rating'].apply(lambda x: None if x is None else 
                                      None if len(x) == 0 else max(x))
df['rate_min'] = df['rating'].apply(lambda x: None if x is None else 
                                      None if len(x) == 0 else min(x))
df['rate_mean'] = df['rating'].apply(lambda x: None if x is None else 
                                      None if len(x) == 0 else np.mean(list(filter(None, x))))

In [642]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        377185 non-null  object 
 1   propertyType  373707 non-null  object 
 2   street        369978 non-null  object 
 3   city          377185 non-null  object 
 4   sqft          365652 non-null  float64
 5   zipcode       377183 non-null  float64
 6   state         377185 non-null  object 
 7   target        374704 non-null  float64
 8   Price/sqft    363651 non-null  float64
 9   Distance      377185 non-null  object 
 10  rating        377185 non-null  object 
 11  dist_max      372900 non-null  float64
 12  dist_min      372900 non-null  float64
 13  dist_mean     372900 non-null  float64
 14  rate_max      363595 non-null  float64
 15  rate_min      363595 non-null  float64
 16  rate_mean     282353 non-null  float64
dtypes: float64(10), object(7)
memory usage: 48.9+ MB